In [ ]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [ ]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

In [ ]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exhange'})

In [ ]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [ ]:
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
nyse_companies = all_companies[(all_companies.exchangeShortName == 'NYSE')]
nasdaq_companies = all_companies[(all_companies.exchangeShortName == 'NASDAQ')]

lse_stocks = lse_companies[(lse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nyse_stocks = nyse_companies[(nyse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nasdaq_stocks = nasdaq_companies[(nasdaq_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

print(f"num LSE companies: {len(lse_companies)}")
print(f"num LSE stocks: {len(lse_stocks)}")
print()
print(f"num NYSE companies: {len(nyse_companies)}")
print(f"num NYSE stocks: {len(nyse_stocks)}")
print()
print(f"num NASDAQ companies: {len(nasdaq_companies)}")
print(f"num NASDAQ stocks: {len(nasdaq_stocks)}")

In [ ]:
exchange = "NASDAQ"
exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

stock_data = {}
failed = []

symbol_queue = queue.Queue()
[symbol_queue.put(s) for s in exchange_stocks.symbol.tolist()]
symbol_queue.qsize()

def worker():
    while True:
        symbol = symbol_queue.get()
        print(f'Working on {symbol}')

        if symbol in stock_data:
            print(f"Already have {symbol}, skipping")
            continue

        try:
            data = fa_utils.get_ticker_data(symbol, api_key)
            stock_data[symbol] = data
            print(f"{symbol} SUCCESS")
        except:
            print(f"{symbol} FAILED")
            failed.append(symbol)

        symbol_queue.task_done()

for x in range(4):
    name = "Thread_" + str(x)
    t = threading.Thread(name=name,target=worker, daemon=True).start()

symbol_queue.join()

print(len(failed))
print(len(stock_data))

with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
    pickle.dump(stock_data, file)

with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
    pickle.dump(failed, file)

In [ ]:
print(len(failed))
print(len(stock_data))

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
    pickle.dump(stock_data, file)

with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
    pickle.dump(failed, file)

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'rb') as file:
    stock_data_2 = pickle.load(file)

print(len(stock_data_2))
display(stock_data_2)